In [10]:
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import numpy as np
import matplotlib.animation as animation

## Double Pendulum

In [11]:
#Variables
g = 9.8; #gravitation
a = 1; #each pendulum's length

In [12]:
def odeFunc(U, t):
    x1, dx1, x2, dx2 = U; #position 1, velocity 1, position 2, velocity 2.

    return [dx1, g*( 1*((x2-x1)/a - x1/a) - (x1/a)), dx2, g*(-(x2-x1)/a)]; #list of "changes" (derivatives of elements of U)

In [13]:
t = np.linspace(0, 10, 1000); #time, timesteps
#U0 = [1/np.sqrt(2), 0, 1/np.sqrt(2),0]
U0 = [0.1, 0, 0.07, 0] #initial U vector: position 1, velocity 1, position 2, velocity 2.

sol = odeint(odeFunc, U0, t) #calculate solution. returns array where each index holds U_t (U at time t). as such, x1 data is the first column, 
#x1 speed is the second column, etc...

In [14]:
plt.plot(t, sol[:,0]) #graphing the x position of the first body over time

## Generalization to n-Pendulum

In [15]:
def odeFunc(U,t):
    x = U[0:n] #for odeint to work, U is defined as: first n elements are positions of n masses, following n elements are velocities

    ddx = [g*((n-1)*((x[1]-x[0])/a - x[0]/a)-x[0]/a)]; #manually insert first acceleration because its calculation is different

    for i in range(1,n-1):
        temp = g*((n-1-i)*((x[i+1]-2*x[i]+x[i-1])/a)-(x[i]-x[i-1])/a) #calculate all accelerations except for first and last masses
        ddx.append(temp)
    
    temp = g*(-(x[n-1]-x[n-2])/a)
    ddx.append(temp) #manually insert last acceleration

    return np.concatenate((U[n:], ddx)) #since y0 has to be of dim 1, U is formatted so first n elements are position and last n are velocities,
    #so in the end, the "derivative list" that you return in odeFunc has first n elements velocities and last n elements accelerations

In [89]:
def initialize(n, x0): #so you dont be monke
    arr = [x0]*n
    arr2 = [0]*n
    return arr + arr2;

def get_y(sol,num): #gets the y values at a given time (num = frame in animation)
    x = np.concatenate(([0], sol[num][:n])); #includes the stationary anchor
    y = [0]
    for pend in range(n):
        y.append(y[-1]-np.sqrt(a*a-np.abs(x[pend+1]-x[pend])**2)) #simple calculation we did for y values
    return np.array(y)

In [94]:
#Variables
n = 20;
g = 9.8; #gravitation
a = 0.076; #pendulum length
t = np.linspace(0, 10, 1000) #time list
x0 = 0.0196
U0 = initialize(n, x0) #first n entries are initial x positions for masses, second n entries are initial velocities

sol = odeint(odeFunc, U0, t) #calculate solution

## Animation:

In [100]:
%matplotlib tk
##initializing animation
fig, ax = plt.subplots() #plots

scale = 2

line1, = ax.plot([-a*n,a*n], [-a*n,a*n], color="blue") #keep in mind the comma. first frame defines scale
scatter1 = plt.scatter([0], [0], color="orange") #initializing plots - first frame

def update(num, line1, scatter1, sol): #update function for funcanimation
    
    x = [0]
    y = []
    for i in range(n): #the x values for the pendulum in current time
        x.append(sol[num][i] * scale)

    y = get_y(sol,num) #y values in current time

    
    line1.set_data(x,y) #connect the joints to draw pendulum

    offsets = []; #because set offsets is stupid
    for i in range(len(x)): 
        offsets.append((x[i],y[i])); #it wants points as inputs, not an x and y list

    scatter1.set_offsets(offsets); #set scatter - the joints

    return [line1, scatter1] 

ani = animation.FuncAnimation(fig, update, len(sol[:,0]), fargs = [line1, scatter1, sol],
                                interval = 5, blit=True); #len(sol) - amount of frames, fargs = extra
                                                        #arguements for update, interval - length of frame